# Tutorial

In [1]:
using OrdinaryDiffEq
using ModelingToolkit
using DataDrivenDiffEq
using LinearAlgebra, Optim
using DiffEqFlux, Flux
using DataFrames


using Random
using Plots
gr()
using CSV


using Pkg

Pkg.status()

┌ Info: DataDrivenDiffEq : OccamNet is available.
└ @ DataDrivenDiffEq /Users/adrocampos/.julia/packages/DataDrivenDiffEq/fivVr/src/DataDrivenDiffEq.jl:168


Status `~/covid19/discovering_missing_terms/Project.toml`
  [336ed68f] CSV v0.10.7
⌃ [2445eb08] DataDrivenDiffEq v0.8.6
  [a93c6f00] DataFrames v1.4.3
  [aae7a2af] DiffEqFlux v1.52.0
  [41bf760c] DiffEqSensitivity v6.79.0
  [0c46a032] DifferentialEquations v7.6.0
  [5789e2e9] FileIO v1.16.0
⌃ [587475ba] Flux v0.13.7
  [7073ff75] IJulia v1.23.3
  [033835bb] JLD2 v0.4.28
⌃ [b2108857] Lux v0.4.34
⌃ [961ee093] ModelingToolkit v8.33.0
⌃ [429524aa] Optim v1.7.3
  [7f7a1694] Optimization v3.9.2
  [36348300] OptimizationOptimJL v0.1.4
  [1dea7af3] OrdinaryDiffEq v6.31.2
⌃ [91a5bcdd] Plots v1.36.1
  [e88e6eb3] Zygote v0.6.49
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ have new versions available and may be upgradable.


### ODE data for simmulation

In [ ]:
rng = Random.default_rng()
u0 = Float32[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan[1], tspan[2], length = datasize)

function trueODEfunc(du, u, p, t)
    true_A = [-0.1 2.0; -2.0 -0.1]
    du .= ((u.^3)'true_A)'
end

prob_trueode = ODEProblem(trueODEfunc, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(), saveat = tsteps))

### This is an Neural ODE

In [ ]:
dudt2 = Lux.Chain(x -> x.^3,
                  Lux.Dense(2, 50, tanh),
                  Lux.Dense(50, 2))
p, st = Lux.setup(rng, dudt2)
prob_neuralode = NeuralODE(dudt2, tspan, Tsit5(), saveat = tsteps)

In [ ]:
function predict_neuralode(p)
  Array(prob_neuralode(u0, p, st)[1])
end

function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss, pred
end

# Do not plot by default for the documentation
# Users should change doplot=true to see the plots callbacks
callback = function (p, l, pred; doplot = true)
  println(l)
  # plot current prediction against data
  if doplot
    plt = scatter(tsteps, ode_data[1,:], label = "data")
    scatter!(plt, tsteps, pred[1,:], label = "prediction")
    display(plot(plt))
  end
  return false
end

pinit = Lux.ComponentArray(p)
callback(pinit, loss_neuralode(pinit)...; doplot=true)

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(optprob,
                                       ADAM(0.05),
                                       callback = callback,
                                       maxiters = 300)

optprob2 = remake(optprob,u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(optprob2,
                                        Optim.BFGS(initial_stepnorm=0.01),
                                        callback=callback,
                                        allow_f_increases = false)

callback(result_neuralode2.u, loss_neuralode(result_neuralode2.u)...; doplot=true)

# Our data

In [2]:
data_dir = "/Users/adrocampos/covid19/synth_data/"
regions = ["2", "3", "5", "10", "15", "20", "30"][1]
mobility_type = ["inv_dist", "border", "neighbor"][2]
initially_recovered = false

false

In [3]:
positions = CSV.File(data_dir * "positions_" * regions * "_regions.csv")
positions = DataFrame(positions)

Row,id,N,density,x,y
,Int64,Int64,Float64,Float64,Float64
1,1,2780,308.973,0.246368,0.714476
2,2,1081,154.375,0.315868,3.86025


In [4]:
file = "1"

"1"

In [5]:
csv_reader = CSV.File(data_dir * "SIR_" * regions * "_regions_" * mobility_type * "_" * file * ".csv")
df = DataFrame(csv_reader)

Row,t,S1,I1,R1,S2,I2,R2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2773.0,7.0,0.0,1078.0,3.0,0.0
2,0.1,2772.93,6.99984,0.0699992,1077.97,2.99992,0.0299996
3,0.2,2772.86,6.9997,0.139997,1077.94,2.99985,0.0599985
4,0.3,2772.79,6.99959,0.209993,1077.91,2.99979,0.0899967
5,0.4,2772.72,6.99951,0.279989,1077.88,2.99974,0.119994
6,0.5,2772.65,6.99945,0.349984,1077.85,2.9997,0.149992
7,0.6,2772.58,6.99942,0.419978,1077.82,2.99967,0.179988
8,0.7,2772.51,6.99942,0.489972,1077.79,2.99964,0.209985
9,0.8,2772.44,6.99945,0.559966,1077.76,2.99963,0.239981


In [6]:
index = range(1,stop=5001,step=50)
df = df[index,:]

Row,t,S1,I1,R1,S2,I2,R2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,2773.0,7.0,0.0,1078.0,3.0,0.0
2,5.0,2769.47,7.02461,3.50337,1076.49,3.00736,1.50088
3,10.0,2765.85,7.11594,7.03573,1074.95,3.03776,3.0112
4,15.0,2762.1,7.27414,10.6305,1073.37,3.09129,4.5425
5,20.0,2758.18,7.49965,14.3211,1071.73,3.16823,6.1064
6,25.0,2754.07,7.79326,18.1415,1070.02,3.26898,7.71469
7,30.0,2749.72,8.15609,22.1259,1068.23,3.39412,9.37944
8,35.0,2745.1,8.58952,26.3093,1066.34,3.54436,11.113
9,40.0,2740.18,9.09517,30.7274,1064.35,3.72052,12.9281


In [7]:
X = Matrix(df[:, [:S1, :I1, :R1]])'
t = df.t

101-element Vector{Float64}:
   0.0
   5.0
  10.0
  15.0
  20.0
  25.0
  30.0
  35.0
  40.0
  45.0
  50.0
  55.0
  60.0
   ⋮
 445.0
 450.0
 455.0
 460.0
 465.0
 470.0
 475.0
 480.0
 485.0
 490.0
 495.0
 500.0

In [8]:
tspan=(t[begin], t[end])
tsteps = range(tspan[1], tspan[2], length = size(t)[1])

u0 = X[:,1]

3-element Vector{Float64}:
 2773.0
    7.0
    0.0

### Neural ODE for our data

In [ ]:
ann = FastChain(
    FastDense(3, 64, tanh), FastDense(64, 64, tanh), FastDense(64, 1)
)

p = [rand(Float32, 3); initial_params(ann)]

function dudt!(du, u, p, t)

    z = ann(u, p[3:end])

    # S, I, R = u
    # β, γ, N = p[1:3]

    N = 2000

    du[1] = -p[1] * u[1] * u[2] / N - z[1] #dS
    du[2] = p[1] * u[1] * u[2] / N - p[2] * u[2] #dI
    du[3] = p[2] * u[2] #dR
end

prob_UODE = ODEProblem(dudt!, u0, tspan, p) ##prob_neuralode

In [ ]:

## Function to train the network
# Define a predictor
function predict(θ, X=X[:, 1], T=t)
    Array(solve(prob_UODE, Vern7(), u0=X, p=θ,
        tspan=(T[1], T[end]), saveat=T,
        abstol=1e-6, reltol=1e-6,
        sensealg=ForwardDiffSensitivity()
    ))
end


# Define parameters for Multiple Shooting
group_size = 5
continuity_term = 200.0f0

function loss(data, pred)
    return sum(abs2, data - pred)
end

function shooting_loss(p)
    return multiple_shoot(p, X, t, prob_UODE, loss, Vern7(),
        group_size; continuity_term)
end

function loss(θ)
    X̂ = predict(θ)
    sum(abs2, X - X̂) / size(X, 2) + convert(eltype(θ), 1e-3) * sum(abs2, θ[3:end]) ./ length(θ[3:end])
end

# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ, args...) = begin
    l = loss(θ) # Equivalent L2 loss
    push!(losses, l)
    if length(losses) % 5 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end

## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS
res1 = DiffEqFlux.sciml_train(shooting_loss, p, ADAM(0.1f0), cb=callback, maxiters=100)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Train with BFGS to achieve partial fit of the data
res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters=500)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Full L2-Loss for full prediction
res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters=100)
println("Final training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
node_preds = predict_neuralode(result_neuralode2.u)

pS = plot(tsteps, [X[1,:], node_preds[1,:]], label=["data S" "prediction S"])
pI = plot(tsteps, [X[2,:], node_preds[2,:]], label=["data I" "prediction I"])
pR = plot(tsteps, [X[3,:], node_preds[3,:]], label=["data R" "prediction R"])

display(plot(pS, pI, pR, layout = (3,1), size = (800, 800)))

## Universal Differential Equations

In [ ]:

ann = FastChain(FastDense(3, 16, tanh),
    # FastDense(50, 50, tanh),
    FastDense(16, 1))

p = [rand(Float32, 3); initial_params(ann)]

function dudt_(du, u, p, t)
    
    S, I, R = u
    β, γ, N  = p[1:3]

    z  = ann(u, p[3:end])
    dS = -β * S * I/N - z[1]  # susceptible
    dI =  β * S * I/N - γ*I  # infected
    dR =  γ * I

    du[1] = dS
    du[2] = dI
    du[3] = dR

end




# Define the problem
prob_UODE = ODEProblem(dudt_, u0, tspan, p) ##prob_neuralode


In [ ]:

# Define parameters for Multiple Shooting
# group_size = 5
# continuity_term = 200.0f0

# function loss1(data, pred)
#     return sum(abs2, data - pred)
# end

# function shooting_loss(p)
#     return multiple_shoot(p, X, t, prob_UODE, loss, Vern7(),
#                           group_size; continuity_term)
# end


function predict(θ)
    Array(solve(prob_UODE, Vern7(), u0=u0, p=θ, tspan=tspan, saveat=t))
end

function loss(θ)
    X̂ = predict(θ)
    sum(abs2, X - X̂) #/ size(X, 2) + convert(eltype(θ), 1e-3)*sum(abs2, θ[3:end]) ./ length(θ[3:end])
end




# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ,args...) = begin
	l = loss(θ) # Equivalent L2 loss
    push!(losses, l)
    if length(losses)%5==0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end

callback(p)

## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS


    
    # Train with BFGS to achieve partial fit of the data
# res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 500)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")

    
    
    # # Full L2-Loss for full prediction
# res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10000)
# println("Final training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
res_fit = DiffEqFlux.sciml_train(loss, p, BFGS(initial_stepnorm=0.1f0), maxiters=2)

In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()
optf    = Optimization.OptimizationFunction((x,p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, p)

result_neuralode = Optimization.solve(optprob, ADAM(0.05), tspan=tspan, saveat=t, maxiters = 10)

In [ ]:
result_neuralode

In [ ]:
preds_uode = Array(result_neuralode)

pS = plot(tsteps, [X[1,:], preds_uode[1,:]], label=["data S" "prediction S"])
pI = plot(tsteps, [X[2,:], preds_uode[2,:]], label=["data I" "prediction I"])
pR = plot(tsteps, [X[3,:], preds_uode[3,:]], label=["data R" "prediction R"])

display(plot(pS, pI, pR, layout = (3,1), size = (800, 800)))

In [ ]:
losses

In [ ]:
## How to plot the loss using Optimization?

In [ ]:
typeof(result_neuralode)

In [ ]:
pred[:,1]

In [ ]:
result_neuralode.t

In [ ]:
pred = reduce(hcat,result_neuralode.u)
t = result_neuralode.t

tspan = (t[begin], t[end])

steps = range(1,87)


# println(tsteps)

# pred = predict(p_trained, X[:,1], tsample)

pS = scatter(pred[1,:], label = "data S")

# # # # scatter!(pS, tsteps, pred[1,:], label = "prediction S")

pI = scatter(pred[2,:], label = "data I")
scatter!(pI, tsteps, pred[2,:], label = "prediction I")
        
# # # # pR = scatter(tsteps, X[3,:], label = "data R")b
# # # # scatter!(pR, tsteps, pred[3,:], label = "prediction R")
   
display(plot(pS, pI, layout = (3,1)))

In [ ]:
t

In [ ]:
## Analysis of the trained network
# Interpolate the solution



In [ ]:
result_neuralode

In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction(loss, adtype)
optprob = Optimization.OptimizationProblem(optf, p)

result_neuralode = Optimization.solve(prob_UODE,
                                       Tsit5(),
#                                        callback = callback,
                                       maxiters = 300)

In [ ]:

function predict(θ)
    Array(concrete_solve(prob_UODE, Vern7(), u0, θ, saveat = t))
end

# No regularisation right now
function loss(θ)
    pred = predict(θ)
    sum(abs2, X .- pred), pred # + 1e-5*sum(sum.(abs, params(ann)))
end

loss(p)

const losses = []
callback(θ,l,pred) = begin
    push!(losses, l)
#     if length(losses)%50==0
#         println(losses[end])
#     end
#     false
end

res1_uode = DiffEqFlux.sciml_train(loss, p, ADAM(0.01), cb=callback, maxiters = 5)
# res2_uode = DiffEqFlux.sciml_train(loss, res1_uode.minimizer, BFGS(initial_stepnorm=0.01), cb=callback, maxiters = 10000)

In [ ]:
# function predict(θ, X=X[:,1], T=t)
#     Array(solve(prob_UODE, Vern7(), u0=X, p=θ, tspan=tspan, saveat=T))
#     end
    
    
    
# function loss(data, pred)
# 	return sum(abs2, data - pred)
# end

    
    
function loss(θ)
    X̂ = predict(θ)
    sum(abs2, Xₙ - X̂) / size(Xₙ, 2) + convert(eltype(θ), 1e-3)*sum(abs2, θ[3:end]) ./ length(θ[3:end])
end

In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, pinit)

result_neuralode = Optimization.solve(prob_UODE,
                                       Adam(0.05),
#                                        callback = callback,
                                       maxiters = 300)

In [ ]:
result_neuralode

In [ ]:
# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ,args...) = begin
	l = loss(θ) # Equivalent L2 loss
    print(l)
    push!(losses, l)
    if length(losses)%5==0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end




In [ ]:
## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS
res1 = DiffEqFlux.sciml_train(shooting_loss, p, ADAM(0.1f0), cb=callback, maxiters = 100)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Train with BFGS to achieve partial fit of the data
res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 500)
println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Full L2-Loss for full prediction
res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10000)
println("Final training loss after $(length(losses)) iterations: $(losses[end])")



In [ ]:
res1 = DiffEqFlux.sciml_train(loss, p, ADAM(0.1f0), cb=callback, maxiters = 100)
println("Training loss after $(length(losses)) iterations: $(losses[end])")



In [ ]:
p_trained = res1.minimizer

In [ ]:



using DiffEqFlux, DifferentialEquations, Optimization, OptimizationOptimJL, Random, Plots, CSV, Lux, DataFrames
using DataDrivenDiffEq, ModelingToolkit, LinearAlgebra, DiffEqSensitivity, Zygote, Optim, CSV, Lux, Pkg, Flux
gr()
Pkg.status()


data_dir = "/Users/adrocampos/covid19/synth_data/"
regions = ["2", "3", "5", "10", "15", "20", "30"][1]
mobility_type = ["inv_dist", "border", "neighbor"][2]
initially_recovered = false

# positions = CSV.File(data_dir * "positions_" * regions * "_regions.csv")
# positions = DataFrame(positions)

file = "1"

csv_reader = CSV.File(data_dir * "SIR_" * regions * "_regions_" * mobility_type * "_" * file * ".csv")
df = DataFrame(csv_reader)


index = range(1, stop=5001, step=50)
df = df[index, :]

X = Matrix(df[:, [:S1, :I1, :R1]])'
t = df.t

tspan = (t[begin], t[end])
tsteps = range(tspan[1], tspan[2], length=size(t)[1])

u0 = X[:, 1]


ann = FastChain(FastDense(3, 16, tanh),
    # FastDense(50, 50, tanh),
    FastDense(16, 3))



## Firs the parameters for Beta, gama und N, then the weigths. 

# Get the initial parameters, first two is linear birth / decay of prey and predator

# Get the initial parameters, first two is linear birth / decay of prey and predator

# ps, st = Lux.setup(rng, ann)

## Firs the parameters for Beta, gama und N, then the weigths. 

# Get the initial parameters, first two is linear birth / decay of prey and predator
p = [rand(Float32, 3); initial_params(ann)]

function dudt_(du, u, p, t)

    S, I, R = u
    β, γ, N = p[1:3]

    # z = ann(u, p[3:end])
    # dS = -β * S * I / N - z[1]  # susceptible
    # dI = β * S * I / N - γ * I - z[1] # infected
    # dR = γ * I

    # du[1] = dS
    # du[2] = dI
    # du[3] = dR

    du[1], du[2], du[3] = ann(u, p[3:end])

end




# Define the problem
prob_UODE = ODEProblem(dudt_, u0, tspan, p) ##prob_neuralode





function predict(θ)
    Array(solve(prob_UODE, Vern7(), u0=u0, p=θ, tspan=tspan, saveat=t))
end

function loss(θ)
    X̂ = predict(θ)
    sum(abs2, X - X̂) / size(X, 2) + convert(eltype(θ), 1e-3) * sum(abs2, θ[3:end]) ./ length(θ[3:end])
end




# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ, args...) = begin
    l = loss(θ) # Equivalent L2 loss
    push!(losses, l)
    if length(losses) % 5 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end

callback(p)






## Training -> First shooting / batching to get a rough estimate

# First train with ADAM for better convergence -> move the parameters into a
# favourable starting positing for BFGS


# res1 = DiffEqFlux.sciml_train(shooting_loss, p, Adam(0.1f0), cb=callback, maxiters=1)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")
# Train with BFGS to achieve partial fit of the data
# res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 500)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")
# # Full L2-Loss for full prediction
# res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters = 10000)
# println("Final training loss after $(length(losses)) iterations: $(losses[end])")

# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()

optf = Optimization.OptimizationFunction((x, p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, p)

result_neuralode = Optimization.solve(optprob, ADAM(0.05), tspan=tspan, saveat=t, maxiters=10)
